In [59]:
import numpy as np
caffe_root = '../../'
import sys
sys.path.insert(0, caffe_root+'python')
import caffe

#net = caffe.Net('../nin_sc_new/deploy.prototxt','../nin_sc_new/cifar10_nin_supercategory_iter_60000.caffemodel', caffe.TEST)
#net = caffe.Net('../super_category_nin/deploy.prototxt','../super_category_nin/cifar10_nin_supercategory_iter_60000.caffemodel', caffe.TEST)
net = caffe.Net('../super_category_finetuning/deploy.prototxt','../super_category_finetuning/nin_sc_iter_7501.caffemodel', caffe.TEST)

#MODEL_FILE = '../super_category_finetuning2/deploy.prototxt'
#PRETRAINED = '../super_category_finetuning2/nin_sc_iter_8000.caffemodel'
# MODEL_FILE = './deploy.prototxt'
# PRETRAINED = '../nin_base.caffemodel'
# net = caffe.Classifier(MODEL_FILE, PRETRAINED,
#                        mean=np.load('./imagenet_mean.npy').mean(1).mean(1),
#                        channel_swap=(2,1,0),
#                        raw_scale=255,
#                        image_dims=(256, 256))
caffe.set_mode_gpu()

In [60]:
def predict(net, inputs, oversample=True):
    """
    Predict classification probabilities of inputs.
    Parameters
    ----------
    inputs : iterable of (H x W x K) input ndarrays.
    oversample : boolean
        average predictions across center, corners, and mirrors
        when True (default). Center-only prediction when False.
    Returns
    -------
    predictions: (N x C) ndarray of class probabilities for N images and C
        classes.
    """
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_mean('data', np.load('./imagenet_mean.npy').mean(1).mean(1)) # mean pixel
    transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
    transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB
    
    if oversample:
        # Generate center, corner, and mirrored crops.
        input_ = caffe.io.oversample(inputs, [224,224])
    else:
        # Take center crop.
        center = np.array([256,256]) / 2.0
        crop = np.tile(center, (1, 2))[0] + np.concatenate([
            np.array([-224,-224]) / 2.0,
            np.array([224,224]) / 2.0
        ])
        input_ = np.array(inputs)[:, crop[0]:crop[2], crop[1]:crop[3], :]

    # Classify
    caffe_in = np.zeros(np.array(input_.shape)[[0, 3, 1, 2]],
                        dtype=np.float32)
    #print caffe_in.shape
    for ix, in_ in enumerate(input_):
        caffe_in[ix] = transformer.preprocess(net.inputs[0], in_)
    out = net.forward_all(data=caffe_in)
    predictions = out['pool4']

    # For oversampling, average predictions across crops.
    if oversample:
        predictions = predictions.reshape((len(predictions) / 10, 10, -1))
        predictions = predictions.mean(1)

    return predictions

In [61]:
labels = open('/storage/datasets/ILSVRC2012/synset_words.txt').readlines()
#labels = [ for label in labels]
#print labels

In [63]:
def run():
    cnt = 0
    correct = 0
    top5_correct = 0
    
    import lmdb
    env = lmdb.open('/storage/datasets/ILSVRC2012/val_lmdb')
    with env.begin() as txn:
        cursor = txn.cursor()
        import time
        start = time.time()
        for key,value in cursor : 
            datum = caffe.proto.caffe_pb2.Datum()
            datum.ParseFromString(value)
            label = datum.label
            temp = np.fromstring(datum.data,dtype=np.uint8).reshape(3,256,256)
            img = np.zeros([256,256,3],np.uint8);
            img[:,:,0] = temp[2,:,:];
            img[:,:,1] = temp[1,:,:];
            img[:,:,2] = temp[0,:,:];
            
            prediction = predict(net, [img], oversample=False).reshape(1000)

            if prediction.argmax() == label :
                correct = correct + 1
            top_5 = prediction.argsort()[-5:][::-1]
            if label in top_5 :
                top5_correct = top5_correct + 1
            cnt = cnt + 1
            if( cnt % 100 == 0 ):
                import sys;
                print "Middle Result : Top-1(%.2f), Top-5(%.2f)"%(float(correct)/float(cnt)*100, float(top5_correct)/float(cnt)*100),
                sys.stdout.flush()
    print "Result : Top-1(%.2f), Top-5(%.2f)"%(float(correct)/float(cnt), float(top5_correct)/float(cnt))
    
run();

Middle Result : Top-1(47.00), Top-5(71.00) Middle Result : Top-1(54.50), Top-5(74.00) Middle Result : Top-1(54.00), Top-5(75.33) Middle Result : Top-1(52.75), Top-5(75.25) Middle Result : Top-1(52.60), Top-5(75.00) Middle Result : Top-1(52.33), Top-5(74.33) Middle Result : Top-1(52.00), Top-5(74.43) Middle Result : Top-1(51.88), Top-5(74.25) Middle Result : Top-1(52.00), Top-5(74.89) Middle Result : Top-1(51.20), Top-5(73.90) Middle Result : Top-1(49.82), Top-5(73.09) Middle Result : Top-1(49.42), Top-5(72.75) Middle Result : Top-1(49.23), Top-5(72.54) Middle Result : Top-1(49.21), Top-5(72.71) Middle Result : Top-1(48.87), Top-5(72.20) Middle Result : Top-1(48.75), Top-5(72.38) Middle Result : Top-1(48.94), Top-5(72.47) Middle Result : Top-1(48.89), Top-5(72.83) Middle Result : Top-1(48.68), Top-5(72.74) Middle Result : Top-1(48.35), Top-5(72.80) Middle Result : Top-1(48.86), Top-5(73.33)

KeyboardInterrupt: 